<a href="https://colab.research.google.com/github/AstridBenamou/M-moire-M1---S09/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Treatment of data


## Import libraries 

In [ ]:
import pandas as pd
import numpy as np
import os 

## Upload data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Data_tennis/Raw_data/data_2000_2020.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (12,15,16,17,18,38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Columns to drop

We drop these columns either because they contain info that are not available before the tennis game takes place or because they contain too many values that would make the number of columns after the one-hot encoding way too big. 

In [ ]:
liste = ['W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5','Wsets', 'Lsets','Location', 'Tournament', 'MaxW',	'MaxL',	'AvgW',	'AvgL']
df = df.drop(liste, axis = 1)

## Get info on dataframe

This function allows us to get a list of the colums with the number of values, the number of null values and their type. 

The argument 'only_empty = True' makes the function send back only columns that have null values

In [ ]:
def get_info_columns(df,only_empty=True):
    df_info=pd.DataFrame()
    cols=[]
    isnull=[]
    count_values=[]
    type_cols=[]
    for col in df.columns:
      if only_empty:
        if df[col].isnull().sum() >0:
          #if col=="Location":
          cols.append(col)
          isnull.append(df[col].isnull().sum())
          count_values.append(len(df[col].value_counts()))
          type_cols.append(df[col].dtype)
          #print("*"*5,'\n',"col:",col,"\nNulles",df[col].isnull().sum(),"\nvalue counts",len(df[col].value_counts()),"\ntype",df[col].dtype,"\n")
      else:
        cols.append(col)
        isnull.append(df[col].isnull().sum())
        count_values.append(len(df[col].value_counts()))
        type_cols.append(df[col].dtype)

    df_info["cols"]=cols
    df_info["isnull"]=isnull
    df_info["count_values"]=count_values
    df_info["type_cols"]=type_cols
    
    return df_info

In [ ]:
info = get_info_columns(df,only_empty=False)

## Sort players by rank to reduce number of values 

20 premiers -> garde leur noms 
20-50 -> "classé-20-50"
50-100 -> "classé-20-50"
100-200 -> "classé-100-200"
200-500 -> "classé-200-500"
500-1000 -> "classé-500-1000"

In [ ]:
df["LRank"] = pd.to_numeric(df["LRank"], errors='coerce')

In [ ]:
df.loc[(df['WRank'] > 15) & (df['WRank'] <= 25),'Winner'] = "classé-15-25"
df.loc[(df['WRank'] > 25) & (df['WRank'] <= 50),'Winner'] = "classé-25-50"
df.loc[(df['WRank'] > 50) & (df['WRank'] <= 100),'Winner'] = "classé-50-100"
df.loc[(df['WRank'] > 100) & (df['WRank'] <= 200),'Winner'] = "classé-100-200"
df.loc[(df['WRank'] > 200) & (df['WRank'] <= 500),'Winner'] = "classé-200-500"
df.loc[(df['WRank'] > 500) & (df['WRank'] <= 1000),'Winner'] = "classé-500-1000"
df.loc[(df['WRank'] >1000),'Winner'] = "classé-plus-de-1000"

In [ ]:
df.loc[(df['LRank'] > 15) & (df['LRank'] <= 25),'Loser'] = "classé-15-25"
df.loc[(df['LRank'] > 25) & (df['LRank'] <= 50),'Loser'] = "classé-25-50"
df.loc[(df['LRank'] > 50) & (df['LRank'] <= 100),'Loser'] = "classé-50-100"
df.loc[(df['LRank'] > 100) & (df['LRank'] <= 200),'Loser'] = "classé-100-200"
df.loc[(df['LRank'] > 200) & (df['LRank'] <= 500),'Loser'] = "classé-200-500"
df.loc[(df['LRank'] > 500) & (df['LRank'] <= 1000),'Loser'] = "classé-500-1000"
df.loc[(df['LRank'] >1000),'Loser'] = "classé-plus-de-1000"

## Player names 


### Create Player 0, Player 1 and winner

Player 1 = player with best rank between winner and loser 

Player 0 = the other player

winner = 0 or 1 according to which player won 

index of columns to use iloc :

Winner : 9                                                 
Loser : 10                                                  
WRank : 11                                                 
Lrank : 12                          

player0 : 42                                               
player1 : 43                                                
winner : 44

In [ ]:
df["player0"] = "0"  
df["player1"] = "0"  
df["winner"] = "0"  

In [ ]:
df_test = df
df_test.loc[(df_test['LRank'] > df_test['WRank']),'player0'] = df_test.loc[(df_test['LRank'] > df_test['WRank']),'Loser']
df_test.loc[(df_test['LRank'] > df_test['WRank']),'player1'] = df_test.loc[(df_test['LRank'] > df_test['WRank']),'Winner']
df_test.loc[(df_test['LRank'] > df_test['WRank']),'winner'] = 1

df_test.loc[(df_test['LRank'] <= df_test['WRank']),'player0'] = df_test.loc[(df_test['LRank'] <= df_test['WRank']),'Winner']
df_test.loc[(df_test['LRank'] <= df_test['WRank']),'player1'] = df_test.loc[(df_test['LRank'] <= df_test['WRank']),'Loser']
df_test.loc[(df_test['LRank'] <= df_test['WRank']),'winner'] = 0

In [ ]:
#df_test.drop(df_test['player0'] == '0' | df_test['player1'] == '0')
df_test.drop(df_test[df_test['player0'] == '0'].index, axis=0, inplace=True)
df_test.drop(df_test[df_test['player1'] == '0'].index, axis=0, inplace=True)

In [ ]:
df = df_test

higher rank (worst player) as player 0  
mieux classé a le plus petit Rank 
for i in range(0,len(df_test)):
  if (df_test.iloc[i,11] < df_test.iloc[i,12]): #WRank < LRank (winner better ranked) # winner -> player1 #loser -> player0 #rand = 1  
    df_test.iloc[i,43] = df_test.iloc[i,9]
    df_test.iloc[i,42] = df_test.iloc[i,10]
    df_test.iloc[i,44] = 1
  else: #winer -> player0 loser -> player1
    df_test.iloc[i,43] = df_test.iloc[i,10]
    df_test.iloc[i,42] = df_test.iloc[i,9]
    df_test.iloc[i,44] = 0

### Create columns to associate variables to player 1 and 0 instead of W and L 

In [ ]:
dfW = df_test[df_test["WRank"] < df_test['LRank']] 
dfW = dfW.rename(columns={"WRank": "RankP1", "LRank": "RankP0",'WPts' : 'PtsP1', 'LPts' :'PtsP0' ,'B365W' : 'B365P1', 'B365L' : 'B365P0', 'CBW' : 'CBP1', 'CBL' : 'CBP0',
       'EXW' : 'EXP1','B&WW': 'B&WP1','B&WL': 'B&WP0', 'EXL' : 'EXP0', 'IWW' : 'IWP1', 'IWL' : 'IWP0', 'PSW': 'PSP1', 'PSL':'PSP0', 'UBW' : 'UBP1', 'UBL': 'UBP0', 'LBW':'LBP1', 'LBL':'LBP0',
       'SJW' : 'SJP1', 'SJL' : 'SJP0', 'MaxW' : 'MaxP1', 'MaxL' : 'MaxP0', 'AvgW' : 'AvgP1', 'AvgL' : 'AvgP0' , 'SBW':"SBP1", 'SBL':'SBP0'})


In [ ]:
dfL = df_test[df_test["WRank"] >= df_test['LRank']] 
dfL = dfL.rename(columns={"WRank": "RankP0", "LRank": "RankP1",'WPts' : 'PtsP0', 'LPts' :'PtsP1' ,'B365W' : 'B365P0', 'B365L' : 'B365P1', 'CBW' : 'CBP0', 'CBL' : 'CBP1',
       'EXW' : 'EXP0', 'B&WW': 'B&WP0','B&WL': 'B&WP1','EXL' : 'EXP1', 'IWW' : 'IWP0', 'IWL' : 'IWP1', 'PSW': 'PSP0', 'PSL':'PSP1', 'UBW' : 'UBP0', 'UBL': 'UBP1', 'LBW':'LBP0', 'LBL':'LBP1',
       'SJW' : 'SJP0', 'SJL' : 'SJP1', 'MaxW' : 'MaxP0', 'MaxL' : 'MaxP1', 'AvgW' : 'AvgP0', 'AvgL' : 'AvgP1' , 'SBW':"SBP0", 'SBL':'SBP1'})

### Now combine the 2 dataframes again 


In [ ]:
dfc = pd.concat([dfW, dfL], ignore_index=True)
df = dfc.sort_values(by="Date")
df = df.drop(['Winner','Loser'], axis = 1)

## Null Values 

### Check columns types and delete na

In [ ]:
df_info = get_info_columns(df)

### To numeric 

In [ ]:
df["winner"] = pd.to_numeric(df["winner"], errors='coerce')
df["EXP0"] = pd.to_numeric(df["EXP0"], errors='coerce')
df["EXP1"] = pd.to_numeric(df["EXP1"], errors='coerce')

### Fill empty spots


In [ ]:
df_info=get_info_columns(df,only_empty = True)
list_col_empty = df_info.cols
for col in list_col_empty:
  df.loc[df[col].isna()==True,col] = df.loc[:,col].mean()

## One Hot encoding

In [ ]:
cols_to_dumb = ['Series', 'Court', 'Surface', 'Round', 'Comment', 'player0','player1']
df_dumb = pd.get_dummies(df, prefix=None, prefix_sep='_', dummy_na=False, columns=cols_to_dumb, drop_first=False, dtype=None)
df = df_dumb

## some variables could be treated without using one hot encoding in the folowing way : 
(We ended up using the other option so this part was not used)

### Series 

1
Grand Slam  +2000 pts 

2
Masters Cup  

3
Masters 1000 +1000pts

4
Masters         

5
ATP500 +500pts

6
ATP250 +250pts 

7
International Gold 

8
International          
               
                
    


In [ ]:
#df[df['Series'] == 3.3174004235605707]
#df = df.drop(32717)

In [ ]:
#df['Series'] = df['Series'].where(df['Series'] != 'Grand Slam', 1)
#df['Series'] = df['Series'].where(df['Series'] != 'Masters Cup', 2)
#df['Series'] = df['Series'].where(df['Series'] != 'Masters 1000', 3)
#df['Series'] = df['Series'].where(df['Series'] != 'Masters', 4)
#df['Series'] = df['Series'].where(df['Series'] != 'ATP500', 5)
#df['Series'] = df['Series'].where(df['Series'] != 'ATP250', 6)
#df['Series'] = df['Series'].where(df['Series'] != 'International Gold', 7)
#df['Series'] = df['Series'].where(df['Series'] != 'International', 8)

In [ ]:
#df['Series'] = pd.to_numeric(df['Series'], errors='coerce', downcast='integer') 

### Round

In [ ]:
#df['Round'] = df['Round'].where(df['Round'] != 'Round Robin', 0)
#df['Round'] = df['Round'].where(df['Round'] != 'The Final', 1)
#df['Round'] = df['Round'].where(df['Round'] != 'Quarterfinals', 3)
#df['Round'] = df['Round'].where(df['Round'] != '4th Round', 4)
#df['Round'] = df['Round'].where(df['Round'] != '3rd Round', 5)
#df['Round'] = df['Round'].where(df['Round'] != '2nd Round', 6)
#df['Round'] = df['Round'].where(df['Round'] != '1st Round', 7)
#df['Round'] = df['Round'].where(df['Round'] != 'other', 8)

In [ ]:
#df = df_Round

## Treatment for the dates 

We divide the date which was in the format day-month-year in three columns and make them numeric. 





In [ ]:
df['Annee'] = df['Date'].astype(str).str[2:4]
df['Mois'] = df['Date'].astype(str).str[6:7]
df['Jour'] = df['Date'].astype(str).str[8:10]

In [ ]:
df["Annee"] = pd.to_numeric(df["Annee"], errors='coerce')
df["Mois"] = pd.to_numeric(df["Mois"], errors='coerce')
df["Jour"] = pd.to_numeric(df["Jour"], errors='coerce')

In [ ]:
df = df.drop(['Date'], axis = 1)

##Write dataframe in csv file 

In [ ]:
df.to_csv('final_data.csv', index=False)  